In [ ]:
import sys
print (sys.version) """this should read as 3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
                    if not return to anacond environment installation steps"""

import landsatxplore as landsatxplore
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer """help (landsatxplore.api)"""
    
import tarfile
import os
from osgeo import gdal
import io
import osr
import numpy as np
import rasterio as rio
%matplotlib notebook

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib import pyplot

%matplotlib inline
from rasterio.plot import show

from itertools import combinations
import fiona

In [ ]:
"""This API has been retired and replaced by the API at https://m2m.cr.usgs.gov, last checked and used 25/04/2021 
API retirement discovered 30/04/2021, replacement script is pending"""

api = API('janemaps','ulster202020') """log in it won't log in without your USGS credentials, username and password"""

landsat5 = api.search(
    dataset='landsat_tm_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='1984-03-01',
    end_date='2013-06-05',
    max_cloud_cover=5
)

landsat7 = api.search(
    dataset='landsat_etm_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='1999-04-15',
    end_date='2013-02-11',
    max_cloud_cover=5
)

landsat8 = api.search(
    dataset='landsat_ot_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='2013-02-11',
    end_date='2020-04-25',
    max_cloud_cover=5
)
    
    
print('{} landsat5 scenes found.'.format(len(landsat5)))

for scene in landsat5:
    print(scene['summary'])
    
print('{} landsat7 scenes found.'.format(len(landsat7)))

for rasterls7 in landsat7:
    print(rasterls7['displayId'])
    

print('{} landsat8 scenes found.'.format(len(landsat8)))

for scene in landsat8:
    print(scene['entityId'])

api.logout()


"""indexing landsat scenes by entitiy id for streamlined download process"""

landsat5[0]['entityId'], landsat5[1]['entityId'], landsat5[2]['entityId'], landsat5[3]['entityId'], 
landsat5[4]['entityId'], landsat5[5]['entityId'], landsat5[6]['entityId'], landsat5[7]['entityId'],
landsat5[8]['entityId'], landsat5[9]['entityId'], landsat5[10]['entityId'], landsat5[11]['entityId'],
landsat5[12]['entityId'], landsat5[13]['entityId'], landsat5[14]['entityId'], landsat5[15]['entityId'],
landsat5[16]['entityId'], landsat5[17]['entityId'], landsat5[18]['entityId'], landsat5[19]['entityId'],

landsat7[0]['entityId'], landsat7[1]['entityId'], landsat7[2]['entityId'], landsat7[3]['entityId'],
landsat7[4]['entityId'], landsat7[5]['entityId'], landsat7[6]['entityId'], landsat7[7]['entityId'],
landsat7[8]['entityId'], landsat7[9]['entityId'], landsat7[10]['entityId'], landsat7[11]['entityId'],
landsat7[12]['entityId'], landsat7[13]['entityId'], landsat7[14]['entityId'], landsat7[15]['entityId'], 
landsat7[16]['entityId'], landsat7[17]['entityId'], landsat7[18]['entityId'], landsat7[19]['entityId'],

landsat8[0]['entityId'], landsat8[1]['entityId'], landsat8[2]['entityId'], landsat8[3]['entityId'],
landsat8[4]['entityId'], landsat8[5]['entityId'], landsat8[6]['entityId']

ee = EarthExplorer('janemaps', 'ulster202020')

ee.download(scene_id=landsat5[0]['entityId'],
output_dir='raster/landsat5')

ee.download(scene_id=landsat7[19]['entityId'], 
output_dir='raster/landsat7')

ee.download(scene_id=landsat8[6]['entityId'], 
output_dir='raster/landsat8')

ee.logout()


In [ ]:
newFolder = "raster/qroo19"
os.makedirs(newFolder)
tar = tarfile.open("raster/landsat8/LC08_L1TP_018046_20190411_20190422_01_T1.tar.gz", "r:gz")
tar.extractall(newFolder)
tar.close()

newFolder = "raster/qroo11"
os.makedirs(newFolder)
tar = tarfile.open("raster/landsat7/LE07_L1TP_018046_20110224_20161210_01_T1.tar.gz", "r:gz")
tar.extractall(newFolder)
tar.close()

newFolder = "raster/qroo19"
os.makedirs(newFolder)
tar = tarfile.open("raster/landsat8/LC08_L1TP_018046_20190411_20190422_01_T1.tar.gz", "r:gz")
tar.extractall(newFolder)
tar.close()

In [ ]:
data_dir ="raster/qroo"

fp = os.path.join(data_dir,r"C:\\Users\\jrllo\\OneDrive\\Documents\\GitHub\\egm722\\Assignment\\data\\raster\\qroo84\\landsat5_84.tiff")

raster = rio.open(fp)

# Read the grid values into numpy arrays
R = raster.read(3)
G = raster.read(2)
B = raster.read(1)

# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
Rn = normalize(R)
Gn = normalize(G)
Bn = normalize(B)

print("Normalized bands RGB")
print(Rn.min(), '-', Rn.max(), 'mean:', Rn.mean())
print(Gn.min(), '-', Gn.max(), 'mean:', Gn.mean())
print(Bn.min(), '-', Bn.max(), 'mean:', Bn.mean())


# Read the grid values into numpy arrays
NIR1 = raster.read(4)
NIR2 = raster.read(5)
Ba = raster.read(1)

# Normalize the values using the function that we defined earlier
NIR1n = normalize(NIR1)
NIR2n = normalize(NIR2)
Ban = normalize(Ba)

# Create the composite by stacking
qroo84_451 = np.dstack((NIR1n, NIR2n, Ban))

# Let's see how our color composite looks like
plt.imshow(qroo84_451)

In [ ]:
area_of_interest = pd.read_csv('area_of_interest/area_of_interest.txt')
area_of_interest.head()

In [ ]:
geometry = [Point(xy) for xy in zip(area_of_interest.x, area_of_interest.y)]
area_of_interest = area_of_interest.drop(['x', 'y'], axis=1)
area_gdf = GeoDataFrame(area_of_interest, crs="EPSG:32626", geometry=geometry)
print (area_gdf.head())

In [ ]:
lon_point_aoi = [-87.37800, -87.33240, -87.26720, -87.31450]
lat_point_aoi = [20.35620, 20.33500, 20.45440 -87.26720, 20.47330]

polygon_geom_aoi = Polygon(zip(lon_point_aoi, lat_point_aoi))
crs = {'init': 'epsg:32616'}
aoi_poly = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_aoi])       
print(aoi_poly.geometry)

In [ ]:
aoi_poly.crs

In [ ]:
aoi_poly.to_file('area_of_interest/aoi_poly.shp')
aoi_poly = gpd.read_file('area_of_interest/aoi_poly.shp')

In [ ]:
corala = gpd.read_file('coral/corala.shp')
geometry = [Point(xy) for xy in zip(corala.LONG, corala.LAT)]
corala = corala.drop(['LONG', 'LAT'], axis=1)
coral1_gdf = GeoDataFrame(corala, crs="EPSG:32616", geometry=geometry)

print (coral1_gdf.head())

coralb = gpd.read_file('coral/coralb.shp')
geometry = [Point(xy) for xy in zip(coralb.LONG, coralb.LAT)]
coralb = coralb.drop(['LONG', 'LAT'], axis=1)
coral2_gdf = GeoDataFrame(coralb, crs="EPSG:32616", geometry=geometry)

print (coral2_gdf.head())

coralc = gpd.read_file('coral/coralc.shp')
geometry = [Point(xy) for xy in zip(coralc.LONG, coralc.LAT)]
coralc = coralc.drop(['LONG', 'LAT'], axis=1)
coral3_gdf = GeoDataFrame(coralc, crs="EPSG:32616", geometry=geometry)

print (coral3_gdf.head())

In [ ]:
coral1_gdf = coral1_gdf.drop(['ALTITUDE', 'DEPTH'], axis=1)
coral2_gdf = coral2_gdf.drop(['ALTITUDE', 'DEPTH'], axis=1)
print(coral1_gdf.head()), print(coral2_gdf.head())

In [ ]:
coral1_gdf.crs, coral2_gdf.crs, coral3_gdf.crs 

In [ ]:
rows, cols = coral1_gdf.shape
print('coral1 gdf Number of GPS waypoints:{}'.format(rows)) 
rows, cols = coral2_gdf.shape
print('coral2 gdf Number of GPS waypoints:{}'.format(rows))
rows, cols = coral3_gdf.shape
print('coral3 gdf Number of GPS waypoints: {}'.format(rows))

In [ ]:
"""print column heads for each coral shape, allows easy comparision of colums ensuring ALTITUDE and DEPTH columns
have been dropped"""
coral1_gdf.columns, coral2_gdf.columns, coral3_gdf.columns 

In [ ]:
coral1_gdf.geometry, coral2_gdf.geometry, coral3_gdf.geometry

In [ ]:
lon_point_corala = [-87.312917,- 87.312983, -87.313067, -87.313117, -87.313150, -87.313267, -87.313283, -87.313200, 
-87.313100, -87.313033, -87.312983, -87.312950, -87.312833, -87.312733, -87.312717, -87.312767, -87.312767, -87.312733, 
-87.312767, -87.312833, -87.312833, -87.312867, -87.312700, -87.312667, -87.312583, -87.312533, -87.312467, -87.312417,
-87.312433, -87.312450, -87.312467, -87.312417, -87.312383, -87.312367, -87.312283, -87.312250, -87.312267, -87.312217,
-87.312750, -87.312850, -87.312917, -87.312917, -87.312883, -87.312817, -87.312833, -87.312717, -87.312700, -87.313400,
-87.313233, -87.313133, -87.313000, -87.312900, -87.312883, -87.312900, -87.312950, -87.313283, -87.314083, -87.314067,
-87.314017, -87.314033, -87.314133, -87.314167, -87.314267, -87.314217, -87.314100, -87.314017, -87.314067, -87.314167,
-87.314283, -87.314400, -87.314433, -87.314533, -87.314567, -87.314617, -87.314633, -87.314650, -87.314717, -87.314717,
-87.314733, -87.314800, -87.314933, -87.315133, -87.315317, -87.315433, -87.315517, -87.315600, -87.315650, -87.315633,
-87.315567, -87.315467, -87.315400, -87.315300, -87.314967, -87.314850, -87.314733, -87.314583, -87.314517, -87.314433,
-87.314333, -87.314217, -87.314133]
lat_point_corala = [20.393917, 20.393917, 20.393933, 20.393850, 20.393800, 20.393800, 20.393733, 20.393683, 20.393583,
20.393517, 20.393450, 20.393333, 20.393333, 20.393383, 20.393450, 20.393500, 20.393567, 20.393667, 20.393717, 20.393700,
20.393783, 20.393817, 20.393400, 20.393417, 20.393400, 20.393433, 20.393433, 20.393450, 20.393533, 20.393600, 20.393633,
20.393633, 20.393567, 20.393483, 20.393517, 20.393583, 20.393633, 20.393667, 20.393350, 20.393300, 20.393283, 20.393233,
20.393133, 20.393083, 20.393033, 20.393017, 20.393067, 20.392967, 20.392850, 20.392833, 20.392800, 20.392600, 20.392667,
20.392783, 20.392850, 20.392950, 20.393000, 20.393083, 20.393233, 20.393333, 20.393400, 20.393417, 20.393350, 20.393633,
20.393700, 20.393717, 20.393783, 20.393783, 20.393783, 20.393800, 20.393867, 20.393917, 20.393850, 20.393750, 20.393667,
20.393650, 20.393667, 20.393600, 20.393550, 20.393500, 20.393433, 20.393450, 20.393483, 20.393467, 20.393450, 20.393483,
20.393400, 20.393350, 20.393250, 20.393217, 20.393167, 20.393067, 20.393150, 20.393167, 20.393133, 20.393150, 20.393133,
20.393117, 20.393067, 20.393067, 20.393033]

polygon_geom_corala = Polygon(zip(lon_point_corala, lon_point_corala))
crs = {'init': 'epsg:32616'}
coral1_poly = gpd.GeoDataFrame(index=[0], crs="EPSG:32616", geometry=[polygon_geom_corala])       
print(coral1_poly.geometry)



""" The method works though it produces this warning
C:\Users\jrllo\anaconda3\envs\turtlepython\lib\site-packages\pyproj\crs\crs.py:53: 
FutureWarning: '+init=<authority>:<code>' syntax is deprecated. 
'<authority>:<code>' is the preferred initialization method. 
When making the change, be mindful of axis order changes: 
https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
return _prepare_from_string(" ".join(pjargs))

code needs to be reworked and future proofed """

In [ ]:
lon_point_coralb = [-87.313150, -87.313183, -87.313150, -87.313083, -87.313067, -87.313017, -87.312983, -87.313033,
-87.312950, -87.312933, -87.312950, -87.312900, -87.312883, -87.312950, -87.312983, -87.312883, -87.312850, -87.312783,
-87.312750, -87.312750, -87.312817, -87.312717, -87.312717, -87.312733, -87.312733, -87.312733, -87.312683, -87.312600,
-87.312533, -87.312500, -87.312533, -87.312550, -87.312483, -87.312433, -87.312400, -87.312367, -87.312350, -87.312417,
-87.312467, -87.312533, -87.312517, -87.312467, -87.312400, -87.312367, -87.312350, -87.312300, -87.312317, -87.312367,
-87.312433, -87.312467, -87.312433, -87.312417, -87.312417, -87.312450, -87.312500, -87.312517, -87.312550, -87.312600,
-87.312617, -87.312600, -87.312350, -87.312450, -87.312517, -87.312533, -87.312583, -87.312600, -87.312600, -87.312583, 
-87.312617, -87.312633, -87.312683, -87.312650, -87.312633, -87.312583, -87.312533, -87.312533, -87.312567, -87.312567,
-87.312500, -87.312483, -87.312450, -87.312433, -87.312417, -87.312400, -87.312383, -87.312500, -87.312417, -87.312367,
-87.312333, -87.312283, -87.312317, -87.312333, -87.312300, -87.312250, -87.312200, -87.312167, -87.312250, -87.312283,
-87.312317]
lat_point_coralb = [20.394367, 20.394383, 20.394383, 20.394383, 20.394417, 20.394450, 20.394433, 20.394367, 20.394433,
20.394450, 20.394467, 20.394533, 20.394550, 20.394600, 20.394633, 20.394600, 20.394600, 20.394600, 20.394617, 20.394550,
20.394517, 20.394467, 20.394483, 20.394500, 20.394483, 20.394600, 20.394683, 20.394767, 20.394767, 20.394817, 20.394883,
20.394933, 20.394967, 20.394917, 20.394850, 20.394800, 20.394750, 20.394717, 20.394717, 20.394733, 20.394700, 20.394600,
20.394483, 20.394417, 20.394317, 20.394217, 20.394150, 20.394150, 20.394150, 20.394167, 20.394217, 20.394233, 20.394300,
20.394333, 20.394367, 20.394400, 20.394383, 20.394383, 20.394433, 20.394483, 20.394067, 20.394117, 20.394150, 20.394150,
20.394100, 20.394067, 20.394033, 20.393967, 20.393983, 20.394017, 20.394000, 20.393933, 20.393900, 20.393767, 20.393700,
20.393683, 20.393650, 20.393617, 20.393633, 20.393650, 20.393683, 20.393733, 20.393850, 20.393883, 20.393967, 20.394100,
20.394100, 20.394133, 20.394150, 20.394217, 20.394300, 20.394417, 20.394450, 20.394500, 20.394533, 20.394533, 20.394583,
20.394550, 20.394517]

polygon_geom_coralb = Polygon(zip(lon_point_coralb, lat_point_coralb))
crs = {'init': 'epsg:32616'}
coral2_poly = gpd.GeoDataFrame(index=[0], crs="EPSG:32616", geometry=[polygon_geom_coralb])       
print(coral2_poly.geometry)

""" The method works though it produces this warning
C:\Users\jrllo\anaconda3\envs\turtlepython\lib\site-packages\pyproj\crs\crs.py:53: 
FutureWarning: '+init=<authority>:<code>' syntax is deprecated. 
'<authority>:<code>' is the preferred initialization method. 
When making the change, be mindful of axis order changes: 
https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
return _prepare_from_string(" ".join(pjargs))

code needs to be reworked and future proofed """

In [ ]:
lon_point_coralc = [-87.312917, -87.312867, -87.312800, -87.312800, -87.312967, -87.313000, -87.313033, -87.313050,
-87.313100, -87.313100, -87.313017, -87.312950, -87.312917, -87.312883, -87.312867, -87.312900, -87.312900, -87.312817,
-87.312817, -87.312850, -87.312767, -87.312633, -87.312600, -87.312567, -87.312517, -87.312533, -87.312567, -87.312533,
-87.312517, -87.312517, -87.312633, -87.312667, -87.312733, -87.312800, -87.312817, -87.312917, -87.313017, -87.313083,
-87.313167, -87.313233, -87.313283, -87.313233, -87.313217, -87.313300, -87.313200, -87.312800]
lat_point_coralc = [20.394583, 20.394617, 20.394633, 20.394667, 20.394667, 20.394783, 20.394833, 20.394900, 20.394883,
20.394850, 20.394883, 20.394883, 20.394933, 20.395017, 20.395100, 20.395183, 20.395317, 20.395383, 20.395433, 20.395483,
20.395467, 20.395367, 20.395333, 20.395183, 20.395050, 20.394950, 20.394900, 20.394833, 20.394850, 20.394917, 20.394900,
20.394867, 20.394800, 20.394767, 20.394617, 20.394583, 20.394550, 20.394467, 20.394433, 20.394433, 20.394467, 20.394583,
20.394750, 20.394450, 20.394650, 20.395534]

polygon_geom_coralc = Polygon(zip(lon_point_coralc, lat_point_coralc))
crs = {'init': 'epsg:32616'}
coral3_poly = gpd.GeoDataFrame(index=[0], crs="EPSG:32616", geometry=[polygon_geom_coralc])       
print(coral3_poly.geometry)

""" The method works though it produces this warning
C:\Users\jrllo\anaconda3\envs\turtlepython\lib\site-packages\pyproj\crs\crs.py:53: 
FutureWarning: '+init=<authority>:<code>' syntax is deprecated. 
'<authority>:<code>' is the preferred initialization method. 
When making the change, be mindful of axis order changes: 
https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
return _prepare_from_string(" ".join(pjargs))

code needs to be reworked and future proofed """

In [ ]:
print('coral 1, perimeter: ', coral1_poly.length) 
print('coral 1, area: ', coral1_poly.area) 
print('coral 1, centroid: ', coral1_poly.centroid)
print('coral 1, bounding coordinates: ', coral1_poly.bounds) 
print('coral 1, bounding box: ', coral1_poly.envelope) 

print('coral 2, perimeter: ', coral2_poly.length) 
print('coral 2, area: ', coral2_poly.area)
print('coral 2, centroid: ', coral2_poly.centroid)
print('coral 2, bounding coordinates: ', coral2_poly.bounds)
print('coral 2, bounding box: ', coral2_poly.envelope)

print('coral 3, perimeter: ', coral3_poly.length)
print('coral 3, area: ', coral3_poly.area)
print('coral 3, centroid: ', coral3_poly.centroid)
print('coral 3, bounding coordinates: ', coral3_poly.bounds)
print('coral 3, bounding box: ', coral3_poly.envelope)

In [ ]:
coral1_poly.to_file('coral/coral1_poly.shp')
coral2_poly.to_file('coral/coral2_poly.shp')
coral3_poly.to_file('coral/coral3_poly.shp')

In [ ]:
coral1_poly = gpd.read_file('coral/coral1_poly.shp')
coral2_poly = gpd.read_file('coral/coral2_poly.shp')
coral3_poly = gpd.read_file('coral/coral3_poly.shp')
coral1_poly.crs, coral2_poly.crs, coral3_poly.crs

In [ ]:
SHOW_NEW_POLYGONS = False

polygons, set_of_properties = [], []
with fiona.open("coral/coral1_poly.shp", "r") as source:
    for line in source:
        polygons.append(shape(line["geometry"]))
        set_of_properties.append(line["properties"])
    meta = source.meta

poly_index_combinations = combinations(tuple(range(len(polygons))), 2)
while True:
    intersection_record = []
    for i_corala_poly, i_coral2_poly in poly_index_combinations:
        coral1_poly, coral2_polb = polygons[i_coral1_poly], polygons[i_coral2_poly]
        if coral1_poly.exterior.xy == coral2_poly.exterior.xy:
            # print(f"The polygons have identical exterior coordinates:\n{poly_a} and {poly_b}\n")
            continue

        intersecting = corala_poly.intersection(coral2_poly)
        if type(intersecting) != Polygon:
            continue
        intersecting_polygons = intersecting.exterior.coords
        if not intersecting_polygons:
            # print(f"No intersections between\n{poly_a} and {poly_b}\n")
            continue

        print("Rectifying intersection")
        y_diff = abs(intersecting_polygons[0][1]) - abs(intersecting_polygons[2][1])

        coral_all = Polygon((
            Point(float(coral2_poly.exterior.coords[0][0]), float(coral2_poly.exterior.coords[0][1] + y_diff)),
            Point(float(coral2_poly.exterior.coords[1][0]), float(coral2_poly.exterior.coords[1][1] + y_diff)),
            Point(float(coral2_poly.exterior.coords[2][0]), float(coral2_poly.exterior.coords[2][1] + y_diff)),
            Point(float(coral2_poly.exterior.coords[3][0]), float(coral2_poly.exterior.coords[3][1] + y_diff)),
            Point(float(coral2_poly.exterior.coords[4][0]), float(coral2_poly.exterior.coords[4][1] + y_diff)),
            
            
        ))

        if SHOW_NEW_POLYGONS:
            x, y = coral1_poly.exterior.xy
            plt.plot(x, y)
            x, y = coral2_poly.exterior.xy
            plt.plot(x, y)
            plt.show

        polygons[coral1_poly, coral2_poly] = coral12_poly
        intersection_record.append(True)

    if not intersection_record:
        break

with fiona.open("coral/coral12.shp", "w", **meta) as sink:
    for poly, properties in zip(polygons, set_of_properties):
        sink.write({
            "geometry": mapping(poly),
            "properties": properties
        })


In [ ]:
from itertools import combinations

from shapely.geometry import Polygon, Point, shape, mapping
import matplotlib.pyplot as plt
import fiona

SHOW_NEW_POLYGONS = False

polygons, set_of_properties = [], []
with fiona.open("coral/coral12.shp", "r") as source:
    for line in source:
        polygons.append(shape(line["geometry"]))
        set_of_properties.append(line["properties"])
    meta = source.meta

poly_index_combinations = combinations(tuple(range(len(polygons))), 2)
while True:
    intersection_record = []
    for i_coral12, i_coral3_poly in poly_index_combinations:
        coral12, coral3_poly, coral3_poly = polygons[i_coral12], polygons[i_coral3_poly]
        if coral12.exterior.xy == coral3_poly.exterior.xy:
            # print(f"The polygons have identical exterior coordinates:\n{poly_a} and {poly_b}\n")
            continue

        intersecting = coral12.intersection(coral3_poly)
        if type(intersecting) != Polygon:
            continue
        intersecting_polygons = intersecting.exterior.coords
        if not intersecting_polygons:
            # print(f"No intersections between\n{poly_a} and {poly_b}\n")
            continue

        print("Rectifying intersection")
        y_diff = abs(intersecting_polygons[0][1]) - abs(intersecting_polygons[2][1])

        coral_all = Polygon((
            Point(float(coral3_poly.exterior.coords[0][0]), float(coral3_poly.exterior.coords[0][1] + y_diff)),
            Point(float(coral3_poly.exterior.coords[1][0]), float(coral3_poly.exterior.coords[1][1] + y_diff)),
            Point(float(coral3_poly.exterior.coords[2][0]), float(coral3_poly.exterior.coords[2][1] + y_diff)),
            Point(float(coral3_poly.exterior.coords[3][0]), float(coral3_poly.exterior.coords[3][1] + y_diff)),
            Point(float(coral3_poly.exterior.coords[4][0]), float(coral3_poly.exterior.coords[4][1] + y_diff)),
            
        ))

        if SHOW_NEW_POLYGONS:
            x, y = coral12_poly.exterior.xy
            plt.plot(x, y)
            x, y = coral3_poly.exterior.xy
            plt.plot(x, y)
            plt.show

        polygons[coral12_poly, coral3_poly] = coral12_poly
        intersection_record.append(True)

    if not intersection_record:
        break

with fiona.open("coral/coral123.shp", "w", **meta) as sink:
    for poly, properties in zip(polygons, set_of_properties):
        sink.write({
            "geometry": mapping(poly),
            "properties": properties
        })

In [ ]:
coral123 = gpd.read_file('coral/coral123.shp')
coral123.crs
print('coral123, perimeter: ', coral123.length)
print('coral123, area: ', coral123.area)
print('coral123, centroid: ', coral123.centroid)
print('coral123, bounding coordinates: ', coral123.bounds)
print('coral123, bounding box: ', coral123.envelope)

In [ ]:
id_data = pd.read_csv('turtle/id_data.txt')
noid_data = pd.read_csv('turtle/noid_data.txt')

id_data.columns, noid_data.columns

In [ ]:
id_data['geometry'] = list(zip(id_data['LONG'], id_data['LAT']))
id_data['geometry'] = id_data['geometry'].apply(Point)
noid_data['geometry'] = list(zip(id_data['LONG'], noid_data['LAT']))
noid_data['geometry'] = noid_data['geometry'].apply(Point)

In [ ]:
id_gdf = gpd.GeoDataFrame(id_data)
id_gdf.set_crs("EPSG:32616", inplace=True)
id_gdf = id_gdf.drop(['LONG', 'LAT'], axis=1)
noid_gdf = gpd.GeoDataFrame(noid_data)
noid_gdf.set_crs("EPSG:32616", inplace=True)
noid_gdf = noid_gdf.drop(['LONG', 'LAT'], axis=1)

In [ ]:
id_gdf.crs, noid_gdf.crs

In [ ]:
id_gdf.to_file('turtle/id_points.shp')
noid_gdf.to_file('turtle/noid_points.shp')

rows, cols = id_gdf.shape
print('id_gdf Number of indentified turtle sightings:{}'.format(rows))
rows, cols = noid_gdf.shape
print('noid_gdf Number of unidentified turtle sightings:{}'.format(rows))

In [ ]:
id_points = gpd.read_file('turtle/id_points.shp')
noid_points = gpd.read_file('turtle/noid_points.shp')

In [ ]:
rows, cols = id_points.shape
print('id points columns:{}'.format(cols))
rows, cols = noid_points.shape
print('noid points columns:{}'.format(cols))

In [ ]:
id_points

In [ ]:
id_points[id_points['Name'] == 'FRANKIE']
print('id points FRANKIE:{}'.format(rows))

is_diving = id_points['D_T_C'] =='D'
is_touching = id_points['D_T_C'] =='T'
is_chasing = id_points['D_T_C'] =='C'

diving = id_points[is_diving]
touching = id_points[is_touching]
chasing = id_points[is_chasing]

is_forage = id_points['S_F_R'] =='F'
is_swim = id_points['S_F_R'] =='S'
is_rest = id_points['S_F_R'] =='R'

forage = id_points[is_forage]
swim = id_points[is_swim]
rest = id_points[is_rest]

is_time1 = id_points['Time_code'] =='1'
is_time2 = id_points['Time_code'] =='2'
is_time3 = id_points['Time_code'] =='3'
time1 = id_points[is_time1]
time2 = id_points[is_time2]
time3 = id_points[is_time3]

print(is_diving.head()), print(is_chasing.head()), print(is_touching.head()), 
print(diving.count()), print(touching.count()), print(chasing.count())

print(is_forage.head()), print(is_swim.head()), print(is_rest.head()),
print(forage.count()), print(swim.count()), print(rest.count())

print(is_time1.head()), print(is_time2.head()), print(is_time3.head()),
print(time1.count), print(time2.count), print(time3.count)
